In [1]:
import numpy as np
import json 

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input

In [57]:
from collections import Counter, defaultdict
import os

In [2]:
# upload VGG16
vgg16 = VGG16(weights='imagenet')

2021-09-29 08:19:50.795551: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-29 08:19:50.888214: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-29 08:19:50.967891: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
vgg16.save('vgg16.h5')

In [5]:
#

In [6]:
# preparing the image that can be feed to vgg16 
#
def prepare_image(img_path):
    img = load_img(img_path, target_size=(224,224))
    # Here 224 is the image size that the vgg16 is expecting
    x = img_to_array(img) # convert image to array 
    x = np.expand_dims(x, axis=0) 
    # In order to create a batch of images, 
    # you need an additional dimension: (samples, size1,size2,channels)
    x = preprocess_input(x)
    # adequate your image to the format the model requires.
    return x 

In [7]:
y = prepare_image("Data/toyotaCar1.jpg")
y.shape
# here y is prepared output image that can be fees to vgg16

(1, 224, 224, 3)

In [8]:
preds = vgg16.predict(y)
# here we are asking vgg16 to predict, which is trained on imagenet 
# therefore we have 1000 prediction probablity  
# for 1000 difference images class or index
# probablity is for each index
preds.shape

2021-09-29 08:22:49.237860: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-29 08:22:49.343155: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


(1, 1000)

In [9]:
# preds

In [10]:
CLASS_INDEX = None
# Here we are loading the imagenet class index of 1000 iamge class
CLASS_INDEX_PATH = "Data/imagenet_class_index.json"

In [54]:
# find the top 5 prediction made 
# this will match the prediction with the imagenet_class_index.json file
def get_predictions(preds, top=5):
    global CLASS_INDEX
    #load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    # get the results
    results = []
    for pred in preds:
        # take the last top 5 sorted index which have highest value
        top_indices = pred.argsort()[-top:][::-1]
        # creates tuple of last top 5 sorted prediction and its imagenet_class_index
        result = [tuple(CLASS_INDEX[str(i)])+(pred[i],) for i in top_indices]
        # take top 5 elements and output the last value at index[2]
        result.sort(key=lambda x : x[2], reverse=True)
        results.append(result)
    return results

In [56]:
print(get_predictions(preds,top=5))

[[('n02814533', 'beach_wagon', 0.2752038), ('n04285008', 'sports_car', 0.23623085), ('n02974003', 'car_wheel', 0.2011139), ('n03930630', 'pickup', 0.1662285), ('n03459775', 'grille', 0.042428244)]]


In [59]:
#

In [ ]:
def get_car_categories():
    d = defaultdict(float)
    img_list = os.listdir()